In [1]:
%load_ext sql

In [2]:
import os

In [3]:
host = "localhost"
database = "pizza_runner"
user = os.environ.get('DB_USER')
password = os.environ.get('DB_PASS')

In [4]:
connection_string = f"postgresql://{user}:{password}@{host}/{database}"

In [5]:
print('DB_USER')
print('DB_PASS')

DB_USER
DB_PASS


In [ ]:
%sql $connection_string

In [ ]:
%%sql

SELECT table_name--, column_name, data_type 
FROM information_schema.tables 
WHERE table_schema='public';


SELECT table_name, column_name, data_type 
FROM information_schema.columns 
WHERE table_name IN (
'pizza_recipes',
'runners',
'customer_orders',
'runner_orders',
'pizza_names',
'pizza_toppings')
ORDER BY table_name

# DATASET CLEAN UP

In [ ]:
%%sql

UPDATE customer_orders
SET exclusions = NULL 
WHERE exclusions = '' OR exclusions = 'none';


UPDATE customer_orders
SET extras = NULL
WHERE extras = '' OR extras = 'NaN' OR extras = 'null';

UPDATE runner_orders
SET duration = NULL, distance = NULL, pickup_time = NULL
WHERE duration = 'null' OR distance = 'null' OR pickup_time = 'null';

UPDATE runner_orders
SET cancellation = NULL
WHERE cancellation = 'null' OR cancellation = '';

UPDATE runner_orders
SET duration = REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(duration, ' minutes', ''), ' mins', ''), 'mins', ''), ' minute', ''), 'minutes', '');

UPDATE runner_orders
SET distance = REPLACE(REPLACE(distance, ' km', ''), 'km', '');


# PIZZA METRICS

#### How many pizzas were ordered?

In [ ]:
%%sql

SELECT COUNT(order_id) AS total_pizzas
FROM customer_orders;

#### How many unique customer orders were made?

In [ ]:
%%sql

SELECT COUNT(DISTINCT order_id) AS unq_orders
FROM customer_orders;


#### How many successful orders were delivered by each runner?

In [ ]:
%%sql

SELECT runner_id, COUNT(*) AS deliveries
FROM runner_orders
-- subquery for cancelled orders
WHERE order_id IN (
	SELECT order_id
	FROM runner_orders
	WHERE cancellation IS NULL)
GROUP BY runner_id;

#### How many of each type of pizza was delivered?

In [ ]:
%%sql

SELECT pn.pizza_name, COUNT(co.pizza_id) AS delivery_cnt
FROM customer_orders co
-- left join pizza_name table for name instead of id
LEFT JOIN pizza_names pn 
USING(pizza_id)
WHERE order_id IN (
	SELECT order_id
	FROM runner_orders
	WHERE cancellation IS NULL)
GROUP BY pn.pizza_id, pn.pizza_name;

#### How many Vegetarian and Meatlovers were ordered by each customer?

In [ ]:
%%sql

SELECT co.customer_id, pn.pizza_name, COUNT(co.pizza_id) AS customer_cnt
FROM pizza_names pn
-- left, right or full outer join all would return same results
INNER JOIN customer_orders co 
USING(pizza_id)
GROUP BY customer_id, pn.pizza_name 
ORDER BY pn.pizza_name, COUNT(co.pizza_id) DESC;

#### What was the maximum number of pizzas delivered in a single order?

In [ ]:
%%sql

SELECT MAX(order_id) AS max_pizza_delivered
FROM customer_orders 
-- subquery in where clause to get count of each order by id
WHERE order_id IN (
	SELECT COUNT(order_id)
	FROM customer_orders
	RIGHT runner_orders ro 
	USING(order_id)
	WHERE cancellation IS NULL
	GROUP BY order_id);

#### For each customer, how many delivered pizzas had at least 1 change and how many had no changes?

In [ ]:
%%sql

-- using the coalesce function on exclusions/extras to combine into 1 column, to find pizzas with or without nulls
-- they could then be counted as having been changed or not

SELECT customer_id, COUNT(COALESCE(exclusions, extras)) AS pizzas_changed, 
COUNT(*) - COUNT(COALESCE(exclusions, extras)) AS no_change
FROM customer_orders
WHERE order_id IN (
	SELECT order_id
	FROM runner_orders
	WHERE cancellation IS NULL)
GROUP BY customer_id
ORDER BY customer_id; 

#### How many pizzas were delivered that had both exclusions and extras?

In [ ]:
%%sql

SELECT COUNT(*) AS dbl_changes
FROM customer_orders
WHERE order_id IN (
	SELECT order_id
	FROM runner_orders
	WHERE cancellation IS NULL)
AND (exclusions IS NOT NULL AND extras IS NOT NULL);

#### What was the total volume of pizzas ordered for each hour of the day?

In [ ]:
%%sql

-- using the extract function to pull the hour from the order_time we can then count the rows

SELECT EXTRACT(HOUR FROM order_time) AS order_hour, COUNT(*) AS pizza_vol
FROM customer_orders
WHERE order_id IN (
	SELECT order_id
	FROM runner_orders
	WHERE cancellation IS NULL)
GROUP BY EXTRACT(HOUR FROM order_time)
ORDER BY EXTRACT(HOUR FROM order_time);

#### What was the volume of orders for each day of the week?

In [ ]:
%%sql

-- use case when statement to extract day of week and assign weekday abbr text to number associated with DOW
-- simplier query would be simply to extract the day of week and leave with the number associated with day of week

SELECT
CASE 
	WHEN EXTRACT(DOW FROM order_time) = 1 THEN 'Sun'
	WHEN EXTRACT(DOW FROM order_time) = 2 THEN 'Mon'
	WHEN EXTRACT(DOW FROM order_time) = 3 THEN 'Tue'
	WHEN EXTRACT(DOW FROM order_time) = 4 THEN 'Wed'
	WHEN EXTRACT(DOW FROM order_time) = 5 THEN 'Thur'
	WHEN EXTRACT(DOW FROM order_time) = 6 THEN 'Fri'
	WHEN EXTRACT(DOW FROM order_time) = 7 THEN 'Sat'
END AS day_of_week,
 COUNT(*) AS pizza_vol
FROM customer_orders
WHERE order_id IN (
	SELECT order_id
	FROM runner_orders
	WHERE cancellation IS NULL)
GROUP BY day_of_week;

# RUNNER AND CUSTOMER EXPERIENCE

#### How many runners signed up for each 1 week period? (i.e. week starts 2021-01-01)

In [ ]:
%%sql

-- count runner_id with window function to get cummulative sum, and order with case statement as year starts within 
-- the week of 53
SELECT DISTINCT week, COUNT(r1.runner_id) OVER(ORDER BY 
	CASE WHEN week = 53 THEN 1
		WHEN week = 1 THEN 2
		WHEN week = 2 THEN 3
	END) AS total_cnt
-- extract week from registration date and add interval 1 day as default week not consistant
FROM (SELECT runner_id, (EXTRACT(WEEK FROM (registration_date + INTERVAL '1 day'))) AS week
		FROM runners) AS r1
JOIN runners AS r2
USING(runner_id)
ORDER BY total_cnt;

#### What was the average time in minutes it took for each runner to arrive at the Pizza Runner HQ to pickup the order?

In [ ]:
%%sql

SELECT runner_id,
-- cast pickup_time as timestamp and trunc at the minute location before averaging 
-- the time taken to arrive at Pizza Runner
AVG(DATE_TRUNC('minute', ro.pickup_time::timestamp - co.order_time)) AS avg_arrival
FROM customer_orders co 
INNER JOIN runner_orders ro 
USING(order_id)
WHERE order_id IN (
	SELECT order_id
	FROM runner_orders
	WHERE cancellation IS NULL)
GROUP BY runner_id;

#### Is there any relationship between the number of pizzas and how long the order takes to prepare?

In [ ]:
%%sql

SELECT order_id,
COUNT(order_id) AS pizza_count,
-- use similar method as above to calculate ready time, and average time per pizza
ro.pickup_time::timestamp - co.order_time AS ready_time,
DATE_TRUNC('minute', ro.pickup_time::timestamp - co.order_time) / COUNT(order_id) AS avg_per_pizza
FROM customer_orders co 
INNER JOIN runner_orders ro 
USING(order_id)
WHERE order_id IN (
	SELECT order_id
	FROM runner_orders
	WHERE cancellation IS NULL)
GROUP BY order_id, pickup_time, order_time

-- ** There does seem to be a correlation be the number of pizzas and how long it takes to prepare
-- each pizza averaging around 10 mins;

#### What was the average distance travelled for each customer? (km)

In [ ]:
%%sql

SELECT customer_id, ROUND(AVG(distance::NUMERIC),2) AS avg_distance
FROM runner_orders ro
LEFT JOIN customer_orders co 
USING(order_id)
GROUP BY customer_id
ORDER BY customer_id; 

#### What was the difference between the longest and shortest delivery times for all orders? (mins)

In [ ]:
%%sql

-- calculate duration differance then use concat function to append 'mins' at the end
SELECT CONCAT(MAX(duration::NUMERIC) - MIN(duration::NUMERIC),' mins') AS diff_duration
FROM runner_orders;

#### What was the average speed for each runner for each delivery and do you notice any trend for these values?

In [ ]:
%%sql

SELECT runner_id, ROUND(AVG(duration::NUMERIC),2), COUNT(order_id)
FROM runner_orders
GROUP BY runner_id 

-- ** Runner 3 has a shorter delivery time, however has 1/2 the order count since they started later. ;

#### What is the successful delivery percentage for each runner?

In [ ]:
%%sql 

-- use a cte and a case when tablw to sum the total completed orders, col as success
WITH orders AS (
SELECT runner_id,
SUM(CASE WHEN cancellation IS NULL THEN 1
ELSE 0 END) AS success, COUNT(order_id) AS total
FROM runner_orders
GROUP BY runner_id
)
-- then divid the successful deliveries by total (multiply by 100), and group by runner_id
SELECT runner_id, success, total, ROUND((success::NUMERIC / total::NUMERIC),2) * 100 AS succ_perc
FROM orders
ORDER BY runner_id;

# PRICING AND OPTIMIZATION

#### What are the standard ingredients for each pizza?

In [ ]:
%%sql

-- create new table to fix the pizza_recipe table from horizontal text to vertical numeric for better use

-- DROP TABLE IF EXISTS p_recipe;
CREATE TABLE p_recipe (
	pizza_id int,
	toppings int
);
INSERT INTO p_recipe
(pizza_id, toppings)
VALUES
('1', '1'),
('1', '2'),
('1', '3'),
('1', '4'),
('1', '5'),
('1', '6'),
('1', '8'),
('1', '10'),
('2', '4'),
('2', '6'),
('2', '7'),
('2', '9'),
('2', '11'),
('2', '12');

-- use string_agg to combine all applicable toppings in single row with a ', '
SELECT pizza_name, STRING_AGG(topping_name, ', ') AS toppings
FROM pizza_names n
INNER JOIN p_recipe r
ON r.pizza_id = n.pizza_id
INNER JOIN pizza_toppings t
ON r.toppings = t.topping_id
GROUP BY pizza_name;

#### What was the most commonly added extra?

In [ ]:
%%sql

WITH split_extras AS (
SELECT SPLIT_PART(extras, ',', 1) AS extra_id
FROM customer_orders
WHERE SPLIT_PART(extras, ',', 1) IS NOT NULL
UNION ALL
SELECT SPLIT_PART(extras, ',', 2)
FROM customer_orders 
WHERE SPLIT_PART(extras, ',', 2) IS NOT NULL)
SELECT topping_name, COUNT(extra_id)::NUMERIC AS most_common
FROM split_extras s
INNER JOIN pizza_toppings t
ON s.extra_id::NUMERIC = t.topping_id
WHERE extra_id != ''
GROUP BY topping_name
ORDER BY most_common DESC
LIMIT 1;

#### What was the most common exclusion?

In [ ]:
%%sql

WITH split_exclusions AS (
SELECT SPLIT_PART(exclusions, ',', 1) AS exclusions_id
FROM customer_orders
WHERE SPLIT_PART(exclusions, ',', 1) IS NOT NULL
UNION ALL
SELECT SPLIT_PART(exclusions, ',', 2)
FROM customer_orders
WHERE SPLIT_PART(exclusions, ',', 2) IS NOT NULL
AND SPLIT_PART(exclusions, ',', 2) != '')
SELECT topping_name, COUNT(exclusions_id)::NUMERIC AS most_common
FROM split_exclusions s
INNER JOIN pizza_toppings t
ON s.exclusions_id::NUMERIC = t.topping_id
WHERE exclusions_id != ''
GROUP BY topping_name
ORDER BY most_common DESC
LIMIT 1;

#### Generate an order item for each record in the customers_orders table in the format of one of the following:
##### Meat Lovers
##### Meat Lovers - Exclude Beef
##### Meat Lovers - Extra Bacon
##### Meat Lovers - Exclude Cheese, Bacon - Extra Mushroom, Peppers

#### I don't know how to do this one, suggestions please?

In [ ]:
%%sql



#### Generate an alphabetically ordered comma separated ingredient list for each pizza order from the customer_orders table and add a 2x in front of any relevant ingredients
##### For example: "Meat Lovers: 2xBacon, Beef, ... , Salami"

In [ ]:
%%sql

-- I dont know how to do the 2x, or to put the ingredients alphabetically. Suggestions?

SELECT co.order_id,
STRING_AGG(topping_name, ', ')
FROM pizza_names n
INNER JOIN p_recipe r
ON r.pizza_id = n.pizza_id
INNER JOIN pizza_toppings t
ON r.toppings = t.topping_id
INNER JOIN customer_orders co 
ON co.pizza_id = n.pizza_id
GROUP BY co.order_id
ORDER BY co.order_id;

#### What is the total quantity of each ingredient used in all delivered pizzas sorted by most frequent first?

#### help?

In [ ]:
%%sql



# PRICING AND RATINGS

#### If a Meat Lovers pizza costs 12 dollars and Vegetarian costs 10 dollars and there were no charges for changes how much money has Pizza Runner made so far if there are no delivery fees?

In [ ]:
%%sql 

WITH type_cnt AS (
SELECT pizza_id, COUNT(*) AS pizza_type_cnt
FROM customer_orders ro
WHERE order_id IN (
	SELECT order_id
	FROM runner_orders
	WHERE cancellation IS NULL)
GROUP BY pizza_id
)
SELECT 
SUM(CASE WHEN pizza_id = 1 THEN pizza_type_cnt * 12 
ELSE pizza_type_cnt * 10 END) AS profit
FROM type_cnt t;

#### What if there was an additional 1 dollar charge for any pizza extras?
##### - Add cheese is 1 dollar extra

#### I thought I knew a way to do this with another cte but not sure. sigh

In [ ]:
%%sql



#### The Pizza Runner team now wants to add an additional ratings system that allows customers to rate their runner, 
#### how would you design an additional table for this new dataset? 
#### - generate a schema for this new table and insert your own data for ratings for each successful customer order between 1 to 5.

#### I don't know how to get the ratings I created and the already established columns to line up. 
#### Do I have to assign the ratings to a key in an already created table?

In [ ]:
%%sql

CREATE TABLE customer_reviews (
customer_id INT,
order_id INT, 
runner_id INT, 
rating INT, 
order_time VARCHAR, 
pickup_time VARCHAR
)

INSERT INTO customer_reviews 
(rating)
VALUES
('4'),
('5'),
('3'),
('5'),
('3'),
('4'),
('5'),
('4')

INSERT INTO customer_reviews
(customer_id, order_id, runner_id, order_time, pickup_time)
SELECT co.customer_id, co.order_id, r.runner_id, co.order_time, r.pickup_time
FROM customer_orders co
INNER JOIN runner_orders r
USING(order_id)
WHERE r.cancellation IS NULL
GROUP BY co.customer_id, co.order_id, r.runner_id, co.order_time, r.pickup_time;

#### Using your newly generated table - can you join all of the information together to form a table which has the following information for successful deliveries?
##### customer_id
##### order_id
##### runner_id
##### rating
##### order_time
##### pickup_time
##### Time between order and pickup
##### Delivery duration
##### Average speed
##### Total number of pizzas

#### Pretty sure I can do this once I have the table above created.

In [ ]:
%%sql



#### If a Meat Lovers pizza was 12 dollars and Vegetarian 10 dollars fixed prices with no cost for extras and each runner is paid 30 cents per kilometre traveled how much money does Pizza Runner have left over after these deliveries?

In [ ]:
%%sql

-- cte for pizza counts by type, and excluding cancellations
WITH type_cnt AS (
SELECT pizza_id, COUNT(*) AS pizza_type_cnt
FROM customer_orders ro
WHERE order_id IN (
	SELECT order_id
	FROM runner_orders
	WHERE cancellation IS NULL)
GROUP BY pizza_id
),
-- second cte for for calculation of all runners pay
runners_pay AS (
SELECT (SUM(distance::NUMERIC)) * .30 AS runner_pay
FROM runner_orders
)
-- from subquery and case statement to total gross profit, an then simple select query to minus pay from gross profit
SELECT ROUND(g_profit - runner_pay,2) AS net_profit
FROM
	(SELECT SUM(CASE WHEN pizza_id = 1 THEN pizza_type_cnt * 12 
			ELSE pizza_type_cnt * 10 END) AS g_profit
			FROM type_cnt) gp,
runners_pay;